In [ ]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn, randint
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model
from keras.datasets import mnist
from matplotlib import pyplot as plt
import numpy as np

def define_generator(latent_dim):
    inputs = Input(shape=(...)) # generator input 的維度必須等於latent_dim
    
    # Dense output_shape: (7 * 7 * 128)
    # BatchNormalization
    # Activation = LeakyReLU
    # Reshape, image shape: (7, 7, 128)
    h1 = Dense(7*7*128)(inputs)
    h1 = BatchNormalization()(h1)
    h1 = LeakyReLU(alpha=0.2)(h1)
    h1 = Reshape((7,7,128))(h1)
    
    # Deconvolution, output_shape(14, 14, 64)
    # -> filters = 64, kernel_size = (5, 5), strides = (2, 2), padding='same'
    # BatchNormalization
    # Activation = LeakyReLU
    h2 = Conv2DTranspose(..., (..., ...), strides=(..., ...), padding='same')(h1)
    h2 = ...
    h2 = ...

    # Deconvolution, output_shape(28, 28, 32)
    # -> filters = 32, kernel_size = (5, 5), strides = (2, 2), padding='same'
    # BatchNormalization
    # Activation = LeakyReLU
    h3 = Conv2DTranspose(..., (..., ...), strides=(..., ...), padding='same')(h2)
    h3 = ...
    h3 = ...
    
    # Deconvolution, output_shape(28, 28, 1)
    # -> filters = 1, kernel_size = (5, 5), activation='sigmoid', padding='same'
    outputs = Conv2DTranspose(..., (..., ...), activation=..., padding='same')(h3)
    
    model = Model(inputs, outputs)
    return model

def define_discriminator():
    inputs = Input(shape=(28,28,1)) # discriminator input的維度必須等於generator output(圖片的大小)
    
    # Convolution, output_shape(14, 14, 32)
    # -> filters = 32, kernel_size = (5, 5), strides = (2, 2)
    # Activation = LeakyReLU
    # Dropout
    h1 = Conv2D(32, (5, 5), strides=(2, 2), padding='same')(inputs)
    h1 = LeakyReLU(alpha=0.2)(h1)
    h1 = Dropout(0.4)(h1)
    
    # Convolution, output_shape(7, 7, 64)
    # -> filters = 64, kernel_size = (5, 5), strides = (2, 2)
    # Activation = LeakyReLU
    # Dropout
    h2 = Conv2D(..., (..., ...), strides=(..., ...), padding='same')(h1)
    h2 = ...
    h2 = ...
    
    # Convolution, output_shape(4, 4, 128)
    # -> filters = 128, kernel_size = (5, 5), strides = (2, 2)
    # Activation = LeakyReLU
    # Dropout
    h3 = Conv2D(..., (..., ...), strides=(..., ...), padding='same')(h2)
    h3 = ...
    h3 = ...
    
    # Flatten 坦平
    # Dense 成一個 output
    x = Flatten()(h3)
    outputs = Dense(..., activation='sigmoid')(x)  # discriminator output 的維度等於 1
    
    model = Model(inputs, outputs)
    opt = Adam(0.0002, 0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def define_gan(generator, discriminator):
    discriminator.trainable = False
    gen_noise = generator.input
    gen_output = generator.output
    gan_output = discriminator(gen_output)
    model = Model(gen_noise, gan_output)
    opt = Adam(0.0002, 0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

def load_real_samples():
    img_rows, img_cols = 28, 28
    img_channels = 1
    (x_train, _), (_, _) = mnist.load_data()
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, img_channels)
    x_train = x_train.astype('float32')
    x_train /= 255
    return x_train

def generate_real_samples(dataset, n):
    images = dataset
    ix = randint(0, images.shape[0], n)
    x = images[ix]
    y = ones((n, 1))
    return x, y

# 生成latent_space中的點作為生成器的輸入
def generate_latent_points(latent_dim, n):
    # 在latent_space中生成點
    x_input = randn(latent_dim * n)
    
    x_input = x_input.reshape(n, latent_dim)
    return x_input

# 用生成器生成 n 個假樣本和類標籤
def generate_fake_samples(generator, latent_dim, n):
    # 在letent_space中生成點
    x_input = generate_latent_points(latent_dim, n)
    # 預測輸出值
    x = generator.predict(x_input)
    # 創建假標籤
    y = zeros((n, 1))
    return x, y

def sample_images(g_model, latent_dim):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, latent_dim))
        gen_imgs = g_model.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs.reshape(-1, 28, 28)[cnt, :,:], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        plt.show()
    
def train(g_model, d_model, gan_model, latent_dim, n_epochs=..., n_batch=..., n_eval=...):
    # 用一半的 batch 來訓練判別器
    half_batch = int(n_batch / 2)
    
    x_train = load_real_samples()
    for i in range(n_epochs):
        d_loss = 0
        g_loss = 0
        for j in range(...):
            x_real, y_real = generate_real_samples(x_train, ...)
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, ...)
            #先訓練discriminator
            d_loss1 = d_model.train_on_batch(x_real,y_real)
            d_loss2 = d_model.train_on_batch(x_fake,y_fake)
            d_loss += (d_loss1 + d_loss2)/2

        for j in range(...):
            #生成隨機的值給generator input
            x_gan = generate_latent_points(latent_dim, ...)
            y_gan = ones((...,1))

            #訓練generator騙過discriminator
            g_loss += gan_model.train_on_batch(x_gan, y_gan)
        
        print(i, d_loss, g_loss)
        if (i+1) % n_eval == 0:
            sample_images(g_model, latent_dim)
            
latent_dim = ...
# 建立判別器
discriminator = define_discriminator()
# 建立生成器
generator = define_generator(latent_dim)
# 建立生成對抗網絡
gan_model = define_gan(generator, discriminator)
# 訓練
train(generator, discriminator, gan_model, latent_dim)